# Prospecção de Dados (Data Mining) DI/FCUL - HA2

## Second Home Assignement (MC/DI/FCUL - 2024)

### Fill in the section below

### GROUP:`###`

* Element Name, Number - Hours worked on the project
* Element Name, Number - Hours worked on the project
* Element Name, Number - Hours worked on the project



## Objectives
The purpose of this Home Assignment is:

1. Analyze the itemset/rules generation procedure
2. Process and identify the most relevant rules

**NOTE 1: Students are not allowed to add more cells to the notebook**

**NOTE 2: The notebook must be submited fully executed**


### 1. Read the Dataset

Dataset: Instacart [Market Basket Analysis](https://www.kaggle.com/datasets/psparks/instacart-market-basket-analysis)



The purpose of this Home Assignment is:

1. Analyze the itemset/rules generation procedure
2. Identify the most relevant rules

Please download the HA files from this moodle folder and inspect it like this:
**NOTE 1: Students are not allowed to add more cells to the notebook**

**NOTE 2: The notebook must be submited fully executed**

The dataset has been preprocessed with transactions as lists of integers. It is necessary a decoder to see the actual products bought together

Uncompress the data set (zip file) which will produce 2 files:
* `order_products.pickle` a Python pickle with all the trasactions
* A `products.txt` - decoder table with all the data relative to each item



In [2]:
#Read product names and IDs
lines=open("products.txt", "rt", encoding="utf8").readlines()
products=[0]*len(lines)
for lin in lines[1:]:
    pid, pname, aid, did=lin.strip().split("\t")    
    products[int(pid)]=pname
    
#read transactions
import pickle
orders=pickle.load(open("order_products.pickle", "rb"))

#check names of products on transaction 2 (example):
print("Transaction 2 is", orders[2])
for prod in orders[2]: print("Code: %5d is product: %s" %(prod, products[prod]))

Transaction 2 is [33120, 28985, 9327, 45918, 30035, 17794, 40141, 1819, 43668]
Code: 33120 is product: Organic Egg Whites
Code: 28985 is product: Michigan Organic Kale
Code:  9327 is product: Garlic Powder
Code: 45918 is product: Coconut Butter
Code: 30035 is product: Natural Sweetener
Code: 17794 is product: Carrots
Code: 40141 is product: Original Unflavored Gelatine Mix
Code:  1819 is product: All Natural No Stir Creamy Almond Butter
Code: 43668 is product: Classic Blend Cole Slaw


### Objective 1 - Analyze the itemset/rules generation procedure

1. From the approaches used in classes make a performance analysis up to a threshold level of support
2. Define a good support threshold for analysis according to your computational capabilities


In [63]:
### Add supporting functions here

def getTransactions(orders, products):
    return list(map(lambda i: list(map(lambda prod: products[prod], orders[i])), orders.keys()))

def getSupport(items, Trs):
    N=len(Trs)
    R=[i for i in range(N) if items.issubset(Trs[i])]
    return len(R)/N

def getAllItems(Trs):
    res=set()
    for t in Trs: res=res | t
    return res

def getConfidence(X, Y, Trs):
    # confidence(X=>Y) = support(X U Y)/support(X)
    supp_X=getSupport(X, Trs)
    supp_XUY=getSupport(X|Y, Trs)
    return supp_XUY/supp_X


def getLift(X, Y, Trs):
    num = getSupport(X | Y, Trs)
    den = getSupport(X, Trs) * getSupport(Y, Trs)

    if den == 0:
        return 0

    return num / den
    
def getConviction(X, Y, Trs):
    num = 1 - getSupport(Y, Trs)
    den = 1 - getConfidence(X, Y, Trs)
    
    if den == 0:
        return 0

    return num / den

def print_rule_statistics(A, B, tr_sets):
    X=set(A)
    Y=set(B)
    supp_x=getSupport(X,tr_sets)
    supp_y=getSupport(Y,tr_sets)
    supp_xy=getSupport(X|Y,tr_sets)
    conf_xy=getConfidence(X,Y, tr_sets)
    print("Support of %s = %6.3f" % (list(X),  supp_x))
    print("Support of %s = %6.3f" % (list(Y),  supp_y))
    print("Support of %s = %6.3f" % (list(X)+list(Y),  supp_xy))
    print("Confidence of %s => %s = %6.3f" % (list(X), list(Y),  conf_xy))


from itertools import combinations

def check_support(support_sets, min_support, database):
    D={}
    for ss in support_sets:
        # getSupport is a nasty function
        supp = getSupport(ss, database)
        # Only remains what is above or equals the min_support
        if supp >= min_support: D[ss]=supp
    return D


def combine(support_sets):
    D={}
    SS = list(support_sets.keys())
    for i in range(len(SS)-1):
        for j in range(i+1, len(SS)):
            new_ss = SS[i] | SS[j]
            if len(new_ss)==len(SS[i])+1:
                D[frozenset(new_ss)]=0 # do not initialize support for now
    return D    

def prune(L1, L0):
    #we must check whether all combinations of L1 are in L
    D={}
    for support_set in L1:
        # This combinations functions compute the all the combinations for the sets
        sub_sets = [frozenset(c) for c in combinations(support_set, len(support_set) - 1)] 
        #print("####  ", support_set, sub_sets)
        add_ss=True
        for sub_set in sub_sets:
            # IF the element is not in the previous layer, skip it
            if sub_set not in L0:
                add_ss=False
                break
        if add_ss==True: D[support_set] = L1[support_set]
    return D


In [64]:
### Add processing code here

transactions = getTransactions(orders, products)
tr_sets=[set(tr) for tr in transactions]

# Exemplo de como funciona a getSupport
# p1 = "Organic Egg Whites"
# p2 = "Garlic Powder"

# print("Support for", p1, "is", getSupport(set([p1]), transactions))
# print("Support for", p2, "is", getSupport(set([p2]), transactions))
# print("Support for", p1, "and", p2, "is", getSupport(set([p1, p2]), transactions))

# Exemplo de como funciona a getConfidence
# conf_me=getConfidence({"Organic Egg Whites"}, {"Garlic Powder"}, tr_sets)
# print("Confidence for Organic Egg Whites => Garlic Powder: %6.4f" % conf_me)

# Exemplo de como funciona a getLift e getConviction
# X = {"Organic Egg Whites"}
# Y = {"Garlic Powder"}
# print("Lift for Organic Egg Whites => Garlic Powder: %6.4f" % getLift(X, Y, tr_sets))
# print("Conviction for Organic Egg Whites => Garlic Powder: %6.4f" % getConviction(X, Y, tr_sets))

# Exemplo de como funciona a print_rule_statistics
# A=['Organic Egg Whites','Coconut Butter']
# B=set(['Garlic Powder'])
# print_rule_statistics(A, B, tr_sets)

min_supp=0.6
items=getAllItems(tr_sets)
#getSupport(, tr_sets)
L0 = {frozenset([item]): getSupport({item}, tr_sets) for item in items}
for ss in L0: print(list(ss), " has support =", L0[ss])


KeyboardInterrupt: 

Esta abordagem não deu muito certo

-----------------

In [1]:
### Add supporting functions here
from pyfim import pyeclat
import time
from PD_freqitems import freqitemsets
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
import pandas as pd

def getTransactions(orders, products):
    return list(map(lambda i: list(map(lambda prod: products[prod], orders[i])), orders.keys()))

In [13]:
transactions = getTransactions(orders, products)

transactions[:2]

[['Organic Egg Whites',
  'Michigan Organic Kale',
  'Garlic Powder',
  'Coconut Butter',
  'Natural Sweetener',
  'Carrots',
  'Original Unflavored Gelatine Mix',
  'All Natural No Stir Creamy Almond Butter',
  'Classic Blend Cole Slaw'],
 ['Total 2% with Strawberry Lowfat Greek Strained Yogurt',
  'Unsweetened Almondmilk',
  'Lemons',
  'Organic Baby Spinach',
  'Unsweetened Chocolate Almond Breeze Almond Milk',
  'Organic Ginger Root',
  'Air Chilled Organic Boneless Skinless Chicken Breasts',
  'Organic Ezekiel 49 Bread Cinnamon Raisin']]

In [20]:
tr_enc = TransactionEncoder()
trans_array = tr_enc.fit(transactions).transform(transactions, sparse=True)


In [23]:
df = pd.DataFrame.sparse.from_spmatrix(trans_array, columns=tr_enc.columns_)
df

C:\Users\romul\AppData\Local\Temp\ipykernel_13836\1771378627.py:1: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  df = pd.DataFrame.sparse.from_spmatrix(trans_array, columns=tr_enc.columns_)


,""".5\"""" Waterproof Tape""","""0% Greek, Blueberry on the Bottom Yogurt""","""1% Hydrocortisone Anti-Itch Cream, Tube Anti-Itch""","""1,000 Mg Vitamin C Lemon Lime Effervescent Drink Mix""","""1,000 Mg Vitamin C Orange Effervescent Powdered Drink Mix""","""1,000 Mg Vitamin C Super Orange""","""1,000 Mg Vitamin C Tangerine Grapefruit Effervescent Powdered Drink Mix""","""1,000 mg Vitamin C Effervescent Powder Drink Cranberry Each Pak single""","""1,000 mg Vitamin C Lemon-Lime Flavored Fizzy Drink Mix - 30 PK""","""1-to-1 Baking Flour, Gluten/Wheat/Dairy Free""",...,with Twist Ties Sandwich & Storage Bags,with Xylitol Cinnamon 18 Sticks Sugar Free Gum,with Xylitol Island Berry Lime 18 Sticks Sugar Free Gum,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum,with Xylitol Original Flavor 18 Sticks Sugar Free Gum,with Xylitol Unwrapped Original Flavor 50 Sticks Sugar Free Gum,with Xylitol Unwrapped Spearmint 50 Sticks Sugar Free Gum,with Xylitol Watermelon Twist 18 Sticks Sugar Free Gum,with a Splash of Mango Coconut Water,with a Splash of Pineapple Coconut Water
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3214869,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3214870,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3214871,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3214872,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
FI_apriori=apriori(df, 0.1, use_colnames=True)
FI_apriori 

,support,itemsets
0,0.118030,(Bag of Organic Bananas)
1,0.146993,(Banana)


In [7]:
### Add processing code here
transactions = getTransactions(orders, products)

# tr_enc = TransactionEncoder()
# trans_array = tr_enc.fit(transactions).transform(transactions)
# trans_array

D={"threshold": [0.2, 0.1, 0.05, 0.02, 0.015], 
  #  "num_itemsets":[],
  #  "apriori":[], 
  #  "FP-growth":[], 
   "ECLAT":[], 
   "PD":[]}


for min_supp in D["threshold"]:
    # t0 = time.time()
    # FI_apriori=apriori(transactions_db, min_supp)
    # t1 = time.time()
    # D["num_itemsets"].append(FI_apriori.shape[0])
    # D["apriori"].append(t1-t0)

    # FI_fpg= fpgrowth(transactions_db, min_supp)
    t2 = time.time()
    # D["FP-growth"].append(t2-t1)

    FI_eclat= pyeclat(transactions, min_supp)    
    t3 = time.time()
    D["ECLAT"].append(t3-t2)

    FI_pdfis= freqitemsets(transactions, min_supp)    
    t4 = time.time()
    D["PD"].append(t4-t3)

    
    print(min_supp, 
        #   FI_apriori.shape[0],"\n\tApriori time:", t1-t0, 
        #   "\n\tFP-growth time:",t2-t1,
          "\n\tECLAT time:", t3-t2,
          "\n\tPD time:", t4-t3)
    
df_performance=pd.DataFrame(D)
df_performance

0.2 
	ECLAT time: 29.343719244003296 
	PD time: 14.52272653579712
0.1 
	ECLAT time: 27.512828826904297 
	PD time: 15.40929126739502
0.05 
	ECLAT time: 31.335748434066772 
	PD time: 16.297924757003784
0.02 
	ECLAT time: 30.003934383392334 
	PD time: 19.768900156021118
0.015 
	ECLAT time: 37.02314496040344 
	PD time: 24.684203147888184


,threshold,ECLAT,PD
0,0.200,29.343719,14.522727
1,0.100,27.512829,15.409291
2,0.050,31.335748,16.297925
3,0.020,30.003934,19.768900
4,0.015,37.023145,24.684203


In [6]:
D

{'threshold': [0.2, 0.1, 0.05, 0.02, 0.015],
 'num_itemsets': [],
 'apriori': [],
 'FP-growth': [],
 'ECLAT': [27.650259017944336,
  27.226418018341064,
  28.7149076461792,
  33.18646836280823,
  35.89333128929138],
 'PD': [14.019229173660278,
  14.330294132232666,
  14.705361604690552,
  19.892805337905884,
  24.81515073776245]}

### Your short analysis here

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum

### Objective 2 - Identify the most relevant rules

1. From your predefined support level generate all available itemsets and generate rules
2. Identify a set of 10 relevant rules using the Highest Lift criterion
    * **NOTE**: Present the rules with the product names and not as Integers 
3. Identify the Maximal and Closed Itemsets for the same level of support and generate 5 rules using the Highest Lift
    * **NOTE**: Do not list the Maximal or Closed Itemsets. Present just a few as an example and mention **how many** Closed and Maximal Itemsets were found for the selected support level
    * **NOTE**: Present the rules with the product names and not as Integers 


In [ ]:
### Add supporting functions here



In [ ]:
### Add processing code here



### Your short analysis here

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum
